## Book exercises

### Chapter 13

Exercises 13.1, 13.2, 13.5

## Coding exercises

In [ ]:
from typing import Any, Callable, Sequence, cast

import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize
from IPython.display import HTML
import scipy.stats
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from scipy.stats._distn_infrastructure import (rv_continuous_frozen,
                                               rv_discrete_frozen)

RNG = np.random.default_rng(seed=0)
N_LINSPACE_SAMPLES=100
PLOT_LINE_WIDTH=3

### 1. Inverse transform for continuous random variables

-   Implement a function named `sample_inverse_transform` that takes as inputs the target random variable $X$ and the number of samples to return an array with all drawn samples.

    **Hint:** SciPy's `rv_frozen_continuous`/`rv_frozen_discrete` represent continous/discrete distributions from a known family (e.g., exponential/geometric) after their parameters are fixed.

    The `ppf` function of a random variable $X$ gives $F_X$.

In [ ]:
def sample_inverse_transform(
        rvc_x: rv_continuous_frozen|rv_discrete_frozen, n_samples: int) -> np.ndarray:
    ...

-   Implement a function named `plot_inverse_transform_cont` which calls `sample_inverse_transform`, taking in the same input arguments, and uses the results to plot the probability mass function & cumulative density function of $X$ alongside a histogram over the generated samples.

In [ ]:
def plot_inverse_transform_cont(rvc_x: rv_continuous_frozen, n_samples: int) -> Figure:
    samples = sample_inverse_transform(rvc_x, n_samples)

    ax: Axes
    fig, ax = plt.subplots()
    fig_x_ax = np.linspace(rvc_x.ppf(0.01), rvc_x.ppf(0.99), N_LINSPACE_SAMPLES)
    ax.set_xlim(right=fig_x_ax.max())
    hist, hist_bins = np.histogram(samples)
    rvc_x_pdf_vals = rvc_x.pdf(fig_x_ax)
    norm_fact = hist.max() / rvc_x_pdf_vals.max()
    ax.set_xlabel("t")
    ax.set_ylabel("f(t)")

    _ = ax.plot(fig_x_ax, rvc_x_pdf_vals, color='b', lw=PLOT_LINE_WIDTH, label="$f_X$")
    _ = ax.plot(fig_x_ax, rvc_x.cdf(fig_x_ax), color='g', lw=PLOT_LINE_WIDTH, label="$F_X$")
    _ = ax.stairs(hist / norm_fact, hist_bins, fill=True, color="gray", label="Inv Transf")

    ax.legend(frameon=False)

    return fig

-   Visualize the histogram of 4000 samples drawn to simulate an $Exp(\lambda=0.5)$ instance against the original probability mass function and inspect the output to verify that the inverse transform sampling behaves correctly.

In [ ]:
...

### 2. Inverse transform for discrete random variables

-   Implement a function named `plot_inverse_transform_disc` as the discrete counterpart of `plot_inverse_transform_cont`.

In [ ]:
def plot_inverse_transform_disc(rvd_x: rv_discrete_frozen, n_samples: int) -> Figure:
    samples = sample_inverse_transform(rvd_x, n_samples)

    ax: Axes
    fig, ax = plt.subplots()
    fig_x_ax = np.arange(rvd_x.ppf(0.01), rvd_x.ppf(0.99))
    rvc_x_pmf_vals = rvd_x.pmf(fig_x_ax)
    distr = np.array([(samples == v).sum() for v in fig_x_ax])
    norm_fact = distr.max() / rvc_x_pmf_vals.max()

    ax.set_xlabel("t")
    ax.set_ylabel("p(t)")

    _ = ax.bar(
        fig_x_ax, rvc_x_pmf_vals,
        alpha=0.5, ec='b', lw=PLOT_LINE_WIDTH, fill=False, label="$p_X$")
    _ = ax.plot(fig_x_ax, rvd_x.cdf(fig_x_ax), color='g', lw=PLOT_LINE_WIDTH, label="$F_X$")
    _ = ax.bar(fig_x_ax, distr / norm_fact, alpha=0.5, color="gray", label="Inv Transf")

    ax.legend(frameon=False)

    return fig

-   Visualize the probability distribution of 4000 samples drawn to simulate a $Geometric(p=0.5)$ instance against the original probability density function and inspect the output to verify that the inverse transform sampling behaves correctly.

In [ ]:
...

### 3. Accept-reject for continuous random variables

-   Define a SciPy class for the continuous random variable from example 1 for continuous random variable accept-reject sampling in the lecture.
    
    Namely, the random variable $X$ has $f_X(t)=20t(1-t)^3, \ 0<t<1$.

    **Hint:** Only `__init__` and `_pdf` need to be redefined in the new `Ex1` class with parent `scipy.stats.rv_continuous`.

In [ ]:
class Ex1(scipy.stats.rv_continuous):
    ...

-   Implement a function named `sample_accept_reject_cont` that takes as inputs the target/reference continuous random variables $X/Y$ and the number of accepted samples to return a tuple of two arrays with all drawn samples alongside which of them was accepted; generation should stop after reaching the number of target accepted samples.

    **Hint:** You can use `scipy.otimize.minimize_scalar` to calculate the minimum value of an expression.
    
    You will also need a way of drawing from a $Bernoulli$ distribution to determine whether to accept a reject a sample with some probability.

In [ ]:
def sample_accept_reject_cont(
        rvc_x: rv_continuous_frozen,rvc_y: rv_continuous_frozen,
        n_accepted: int) -> tuple[np.ndarray, np.ndarray]:
    ...

-   Implement a function named `plot_accept_reject_cont` which calls `sample_accept_reject_cont`, taking in the same input arguments, and uses the results to plot the probability density functions of $X/Y$ alongside a histogram over the generated accepted samples.

In [ ]:
def plot_accept_reject_cont(
        rvc_x: rv_continuous_frozen,
        rvc_y: rv_continuous_frozen,
        n_accepted: int) -> Figure:
    samples, accepts = sample_accept_reject_cont(rvc_x, rvc_y, n_accepted)

    ax: Axes
    fig, ax = plt.subplots()
    fig_x_ax = np.linspace(rvc_x.ppf(0.01), rvc_x.ppf(0.99), N_LINSPACE_SAMPLES)
    hist, hist_bins = np.histogram(samples[accepts])
    rvc_x_pdf_vals = rvc_x.pdf(fig_x_ax)
    norm_fact = hist.max() / rvc_x_pdf_vals.max()
    ax.set_xlabel("t")
    ax.set_ylabel("f(t)")

    _ = ax.plot(fig_x_ax, rvc_x_pdf_vals, color='b', lw=PLOT_LINE_WIDTH, label="$f_X$")
    _ = ax.plot(fig_x_ax, rvc_y.pdf(fig_x_ax), color='g', lw=PLOT_LINE_WIDTH, label="$f_Y$")
    _ = ax.stairs(hist / norm_fact, hist_bins, fill=True, color="gray", label="Acc-Rej")

    ax.legend(frameon=False)

    return fig

-   Visualize the histogram of 2000 accepted samples drawn with an `Ex1` instance as the target and a $U(0, 1)$ instance as the reference; inspect the output to verify that the accept-reject sampling mimics the target probability density function.

In [ ]:
...

### 4. Accept-reject for discrete random variables

-   Implement a function named `sample_accept_reject_disc` as the discrete counterpart of `sample_accept_reject_cont`.
    
    **Hint:** You can calculate the value of `c` by directly iterating over all possible values of $Y$ between `ppf(0.01)` and `ppf(0.99)`.

In [ ]:
def sample_accept_reject_disc(
        rvd_x: rv_discrete_frozen, rvd_y: rv_discrete_frozen,
        n_accepted: int) -> tuple[np.ndarray, np.ndarray]:
    ...

-   Implement a function named `plot_accept_reject_disc` as the discrete counterpart of the `plot_accept_reject_cont`.

In [ ]:
def plot_accept_reject_disc(
        rvd_x: rv_discrete_frozen,
        rvd_y: rv_discrete_frozen,
        n_accepted: int) -> Figure:
    samples, accepts = sample_accept_reject_disc(rvd_x, rvd_y, n_accepted)

    ax: Axes
    fig, ax = plt.subplots()
    fig_x_ax = np.arange(rvd_y.ppf(0.01), rvd_y.ppf(0.99))
    rvc_x_pmf_vals = rvd_x.pmf(fig_x_ax)
    accepted_distr = np.array([(samples[accepts] == v).sum() for v in fig_x_ax])
    norm_fact = accepted_distr.max() / rvc_x_pmf_vals.max()

    ax.set_xlabel("t")
    ax.set_ylabel("p(t)")

    _ = ax.bar(
        fig_x_ax, rvc_x_pmf_vals,
        alpha=0.5, ec='b', lw=PLOT_LINE_WIDTH, fill=False, label="$p_X$")
    _ = ax.bar(
        fig_x_ax, rvd_y.pmf(fig_x_ax),
        alpha=0.5, ec='g', lw=PLOT_LINE_WIDTH, fill=False, label="$p_Y$")
    _ = ax.bar(fig_x_ax, accepted_distr / norm_fact, alpha=0.5, color="gray", label="Acc-Rej")

    ax.legend(frameon=False)

    return fig

-   Visualize the histogram of 2000 accepted samples drawn with a $Geometric(p=0.5)$ instance as the target and a $Geometric(p=0.25)$ instance as the reference; inspect the output to verify that the accept-reject sampling mimics the target probability mass function.

In [ ]:
...

### Extra: Animations for continuous random variables

In [ ]:
writer = animation.PillowWriter(fps=10)

-   Animation function for inverse transform sampling of continuous random variables:

In [ ]:
def animate_inverse_transform_cont(
        rvc_x: rv_continuous_frozen,
        n_samples: int) -> tuple[Figure, animation.Animation]:
    samples = sample_inverse_transform(rvc_x, n_samples)

    ax: Axes
    fig, ax = plt.subplots()
    fig_x_ax = np.linspace(rvc_x.ppf(0.), rvc_x.ppf(0.99), N_LINSPACE_SAMPLES)
    ax.set_xlim(right=fig_x_ax.max())
    hist, hist_bins = np.histogram(samples)
    rvc_x_pdf_vals = rvc_x.pdf(fig_x_ax)
    norm_fact = hist.max() / rvc_x_pdf_vals.max()
    first_point, *_ = samples
    ax.set_xlabel("t")
    ax.set_ylabel("f(t)")

    _ =ax.plot(fig_x_ax, rvc_x_pdf_vals, color='b', lw=PLOT_LINE_WIDTH, label="$f_X$")
    _ = ax.plot(fig_x_ax, rvc_x.cdf(fig_x_ax), color='g', lw=PLOT_LINE_WIDTH, label="$F_X$")
    _, _, bar_container = ax.hist(
        [], cast(Sequence[float], hist_bins), ec="white", fc="gray", label="Inv Transf")
    scat = ax.scatter(
        first_point, rvc_x.cdf(first_point),
        s=100, color="black",
        zorder=2, animated=True)

    patches = bar_container.patches
    ax.legend(frameon=False)

    def animate(i: int) -> Any:
        scat.set_offsets((samples[i], rvc_x.cdf(samples[i])))
        n, _ = np.histogram(samples[: i + 1], hist_bins)

        for count, rect in zip(n, patches):
            rect.set_height(count / norm_fact)

        return scat, *patches

    ani = animation.FuncAnimation(
        fig, animate, repeat=True, frames=len(samples), interval=50, blit=True)
    plt.close()

    return fig, ani

-   Write to file and display inverse transform animation (*warning* - long runtime):

In [ ]:
# fig_ani_inverse_transform, ani_inverse_transform = animate_inverse_transform_cont(expon, 250)
# ani_inverse_transform.save("inverse_transform.gif", writer=writer, dpi=250)
# HTML(ani_inverse_transform.to_jshtml())

-   Animation function for accept-reject sampling of continuous random variables:

In [ ]:
def animate_accept_reject_cont(
        rvc_x: rv_continuous_frozen,
        rvc_y: rv_continuous_frozen,
        n_accepted: int) -> tuple[Figure, animation.Animation]:
    samples, accepts = sample_accept_reject_cont(rvc_x, rvc_y, n_accepted)

    ax: Axes
    fig, ax = plt.subplots()
    fig_x_ax = np.linspace(rvc_x.ppf(0.), rvc_x.ppf(0.99), N_LINSPACE_SAMPLES)
    hist, hist_bins = np.histogram(samples[accepts])
    rvc_x_pdf_vals = rvc_x.pdf(fig_x_ax)
    norm_fact = hist.max() / rvc_x_pdf_vals.max()
    first_point, *_ = samples
    ax.set_xlabel("t")
    ax.set_ylabel("f(t)")

    _ =ax.plot(fig_x_ax, rvc_x_pdf_vals, color='b', lw=PLOT_LINE_WIDTH, label="$f_X$")
    _ = ax.plot(fig_x_ax, rvc_y.pdf(fig_x_ax), color='g', lw=PLOT_LINE_WIDTH, label="$f_Y$")
    _, _, bar_container = ax.hist(
        [], cast(Sequence[float], hist_bins), ec="white", fc="gray", label="Acc-Rej")
    scat = ax.scatter(
        first_point, rvc_y.pdf(first_point),
        s=100, edgecolors="black",
        zorder=2, animated=True)

    patches = bar_container.patches
    ax.legend(frameon=False)

    def animate(i: int) -> Any:
        scat.set_offsets((samples[i], rvc_y.pdf(samples[i])))
        scat.set_facecolor("white" if accepts[i] else "red")
        n, _ = np.histogram(samples[: i + 1][accepts[:i + 1]], hist_bins)

        for count, rect in zip(n, patches):
            rect.set_height(count / norm_fact)

        return scat, *patches

    ani = animation.FuncAnimation(
        fig, animate, repeat=True, frames=len(samples), interval=50, blit=True)
    plt.close()

    return fig, ani

-   Write to file and display accept-reject animation (*warning* - long runtime):

In [ ]:
# fig_ani_accept_reject, ani_accept_reject = animate_accept_reject_cont(ex1, uniform, 250)
# ani_accept_reject.save("accept_reject.gif", writer=writer, dpi=250)
# HTML(ani_accept_reject.to_jshtml())